In [1]:
import chipwhisperer as cw

In [2]:
bitstream_path = "C:\\Users\\jhaaa\\Desktop\\cw305_research\\vivado\\soc\\soc.runs\\impl_1\\cw305_top.bit"
scope = cw.scope()
target = cw.target(scope, cw.targets.CW305, bsfile=bitstream_path)

(ChipWhisperer Target WARNING|File CW305.py:591) Using default Verilog defines (c:\Users\jhaaa\Desktop\cw305_research\scripts\my_venv\Lib\site-packages\chipwhisperer/hardware/firmware/cw305/cw305_aes_defines.v); if this is not what you want, provide them via the defines_files argument


In [3]:
target.vccint_set(1.0) #set VCC-INT to 1V
target.pll.pll_outfreq_set(50e6, 1)
target.pll.pll_outfreq_get(0)

9600000.0

In [4]:
# Configure the Scope for capture
scope.gain.db = 40        # Adjust based on signal strength
scope.adc.samples = 5000  # Number of points to capture
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.io.trigger_source = "tio4"
scope.clock.adc_src = "extclk_x4"

(ChipWhisperer Other ERROR|File util.py:419) Setting unknown attribute trigger_source in <class 'chipwhisperer.capture.scopes.cwhardware.ChipWhispererExtra.GPIOSettings'>


In [5]:
target.fpga_write(0x0d, [0x01]) # Mode 0: Encrypt

# 1. Input data as standard NIST order (No manual reversal)
key_hex = "000102030405060700010203040506ff"
pt_hex  = "11112233445566778899aabbccddeed8"

# Convert to list without [::-1]
key = list(bytes.fromhex(key_hex))[::-1]
plaintext = list(bytes.fromhex(pt_hex))[::-1]

print(f"Original PT: {bytes(plaintext).hex(' ')}")

Original PT: d8 ee dd cc bb aa 99 88 77 66 55 44 33 22 11 11


In [6]:
# 2. Encryption
target.fpga_write(0x0d, [0x00]) # Mode 0: Encrypt
target.fpga_write(0x0a, key)    # Key
target.fpga_write(0x06, plaintext) # TextIn 
target.fpga_write(0x05, [0x01]) # Trigger

In [7]:
ciphertext = target.fpga_read(0x09, 16) # CipherOut
print(f"Ciphertext:  {ciphertext.hex(' ')}")

Ciphertext:  19 49 91 1c c5 14 7e 50 5d 67 e7 66 56 2e f7 6b


In [8]:
# 3. Decryption Loopback
target.fpga_write(0x0d, [0x01]) # Mode 1: Decrypt

target.fpga_write(0x07, list(ciphertext)) 
target.fpga_write(0x05, [0x01]) # Trigger

# Based on your Verilog: crypt_textin = aes_mode ? aes_out : 128'b0
# The result should be in REG_CRYPT_TEXTOUT
decrypted_pt = target.fpga_read(0x08, 16)
print(f"Decrypted:   {decrypted_pt.hex(' ')}")

Decrypted:   d8 ee dd cc bb aa 99 88 77 66 55 44 33 22 11 11
